# Imports

In [1]:
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
export_dir = os.getcwd()
from pathlib import Path
import pickle
from collections import defaultdict
import time
import torch
import torch.nn as nn
import copy
import torch.nn.functional as F
import optuna
import logging
import ipynb
import importlib
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import KFold

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
output_type_dict = {
    "VAE":"multiple",
    "MLP":"single",
    "NCF": "single"}

num_users_dict = {
    "ML1M":6037,
    "Yahoo":13797, 
    "Pinterest":19155}

num_items_dict = {
    "ML1M":3381,
    "Yahoo":4604, 
    "Pinterest":9362}

In [3]:
data_name = "ML1M" ### Can be ML1M, Yahoo, Pinterest
recommender_name = "MLP" ## Can be MLP, VAE, NCF

DP_DIR = Path("processed_data", data_name) 
export_dir = Path(os.getcwd()).parent
files_path = Path(export_dir, DP_DIR)
checkpoints_path = Path(export_dir, "checkpoints")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

output_type = output_type_dict[recommender_name] ### Can be single, multiple
num_users = num_users_dict[data_name] 
num_items = num_items_dict[data_name] 

In [4]:
from ipynb.fs.defs.help_functions import *
importlib.reload(ipynb.fs.defs.help_functions)
from ipynb.fs.defs.help_functions import *

## Data imports and preprocessing

In [5]:
train_data = pd.read_csv(Path(files_path,f'train_data_{data_name}.csv'), index_col=0)
test_data = pd.read_csv(Path(files_path,f'test_data_{data_name}.csv'), index_col=0)
static_test_data = pd.read_csv(Path(files_path,f'static_test_data_{data_name}.csv'), index_col=0)
with open(Path(files_path,f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f)
    
train_array = train_data.to_numpy()
test_array = test_data.to_numpy()
items_array = np.eye(num_items)
all_items_tensor = torch.Tensor(items_array).to(device)

In [6]:
for row in range(static_test_data.shape[0]):
    static_test_data.iloc[row, static_test_data.iloc[row,-2]]=0
test_array = static_test_data.iloc[:,:-2].to_numpy()

In [7]:
pop_array = np.zeros(len(pop_dict))
for key, value in pop_dict.items():
    pop_array[key] = value

# Recommenders Import

In [8]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *

# Define the dict

In [9]:
kw_dict = {'device':device,
          'num_items': num_items,
           'num_features': num_items, 
           'demographic':False,
          'pop_array':pop_array,
          'all_items_tensor':all_items_tensor,
          'static_test_data':static_test_data,
          'items_array':items_array,
          'output_type':output_type,
          'recommender_name':recommender_name}

# Training

## MLP Train function

In [10]:
train_losses_dict = {}
test_losses_dict = {}
HR10_dict = {}

def MLP_objective(trial):
    
    lr = trial.suggest_float('learning_rate', 0.001, 0.01)
    batch_size = trial.suggest_categorical('batch_size', [256, 512, 1024])
    hidden_dim = trial.suggest_categorical('hidden_dim', [64, 128, 256, 512])
    beta = trial.suggest_float('beta', 0, 4)
    epochs = 10
    model = MLP(hidden_dim, **kw_dict)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_losses = []
    test_losses = []
    hr10 = []
    print(f'======================== new run - {recommender_name} ========================')
    logger.info(f'======================== new run - {recommender_name} ========================')
    
    num_training = train_data.shape[0]
    num_batches = int(np.ceil(num_training / batch_size))

    
    for epoch in range(epochs):
        train_matrix = sample_indices(train_data.copy(), **kw_dict)
        perm = np.random.permutation(num_training)
        loss = []
        train_pos_loss=[]
        train_neg_loss=[]
        if epoch!=0 and epoch%10 == 0:
            lr = 0.1*lr
            optimizer.lr = lr
        
        for b in range(num_batches):
            optimizer.zero_grad()
            if (b + 1) * batch_size >= num_training:
                batch_idx = perm[b * batch_size:]
            else:
                batch_idx = perm[b * batch_size: (b + 1) * batch_size]    
            batch_matrix = torch.FloatTensor(train_matrix[batch_idx,:-2]).to(device)

            batch_pos_idx = train_matrix[batch_idx,-2]
            batch_neg_idx = train_matrix[batch_idx,-1]
            
            batch_pos_items = torch.Tensor(items_array[batch_pos_idx]).to(device)
            batch_neg_items = torch.Tensor(items_array[batch_neg_idx]).to(device)
            
            pos_output = torch.diagonal(model(batch_matrix, batch_pos_items))
            neg_output = torch.diagonal(model(batch_matrix, batch_neg_items))
            
            pos_loss = torch.mean((torch.ones_like(pos_output)-pos_output)**2)
            neg_loss = torch.mean((neg_output)**2)
            
            batch_loss = pos_loss + beta*neg_loss
            batch_loss.backward()
            optimizer.step()
            
            loss.append(batch_loss.item())
            train_pos_loss.append(pos_loss.item())
            train_neg_loss.append(neg_loss.item())
            
        print(f'train pos_loss = {np.mean(train_pos_loss)}, neg_loss = {np.mean(train_neg_loss)}')    
        train_losses.append(np.mean(loss))
        torch.save(model.state_dict(), Path(checkpoints_path, f'MLP_{data_name}_{round(lr,4)}_{batch_size}_{trial.number}_{epoch}.pt'))


        model.eval()
        test_matrix = np.array(static_test_data)
        test_tensor = torch.Tensor(test_matrix[:,:-2]).to(device)
        
        test_pos = test_matrix[:,-2]
        test_neg = test_matrix[:,-1]
        
        row_indices = np.arange(test_matrix.shape[0])
        test_tensor[row_indices,test_pos] = 0
        
        pos_items = torch.Tensor(items_array[test_pos]).to(device)
        neg_items = torch.Tensor(items_array[test_neg]).to(device)
        
        pos_output = torch.diagonal(model(test_tensor, pos_items).to(device))
        neg_output = torch.diagonal(model(test_tensor, neg_items).to(device))
        
        pos_loss = torch.mean((torch.ones_like(pos_output)-pos_output)**2)
        neg_loss = torch.mean((neg_output)**2)
        print(f'test pos_loss = {pos_loss}, neg_loss = {neg_loss}')
        
        hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR = recommender_evaluations(model, **kw_dict)
        hr10.append(hit_rate_at_10)
        print(hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR)
        
        test_losses.append(-hit_rate_at_10)
        if epoch>5:
            if test_losses[-2]<=test_losses[-1] and test_losses[-3]<=test_losses[-2] and test_losses[-4]<=test_losses[-3]:
                logger.info(f'Early stop at trial with batch size = {batch_size} and lr = {lr}. Best results at epoch {np.argmin(test_losses)} with value {np.min(test_losses)}')
                train_losses_dict[trial.number] = train_losses
                test_losses_dict[trial.number] = test_losses
                HR10_dict[trial.number] = hr10
                return max(hr10)
            
    logger.info(f'Stop at trial with batch size = {batch_size} and lr = {lr}. Best results at epoch {np.argmin(test_losses)} with value {np.min(test_losses)}')
    train_losses_dict[trial.number] = train_losses
    test_losses_dict[trial.number] = test_losses
    HR10_dict[trial.number] = hr10
    return max(hr10)

## VAE Train function

### Define the configs (they are defined once again inside the load recommender function in the "help funcion" notebook

In [11]:
VAE_config= {
"enc_dims": [512,128],
"dropout": 0.5,
"anneal_cap": 0.2,
"total_anneal_steps": 200000}


Pinterest_VAE_config= {
"enc_dims": [256,64],
"dropout": 0.5,
"anneal_cap": 0.2,
"total_anneal_steps": 200000}

### Function

In [12]:
def cross_validate_vae(n_splits=5):
    # Combine data for splitting
    all_data = pd.concat([train_data, test_data])
    
    # Добавляем столбцы pos и neg, если их нет
    if all_data.shape[1] == num_items:
        # Для каждого пользователя находим один случайный положительный и отрицательный предмет
        pos_items = []
        neg_items = []
        for _, user_data in all_data.iterrows():
            # Находим индексы положительных и отрицательных взаимодействий
            pos_indices = np.where(user_data > 0)[0]
            neg_indices = np.where(user_data == 0)[0]
            
            # Выбираем случайный положительный и отрицательный предмет
            if len(pos_indices) > 0:
                pos_items.append(np.random.choice(pos_indices))
            else:
                pos_items.append(0)  # fallback
                
            if len(neg_indices) > 0:
                neg_items.append(np.random.choice(neg_indices))
            else:
                neg_items.append(0)  # fallback
        
        all_data['pos'] = pos_items
        all_data['neg'] = neg_items
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    fold_results = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(all_data)):
        print(f"\nFold {fold + 1}/{n_splits}")
        
        # Split data
        fold_train = all_data.iloc[train_idx]
        fold_test = all_data.iloc[test_idx]
        
        # Создаем копию kw_dict и обновляем static_test_data
        fold_kw_dict = kw_dict.copy()
        fold_kw_dict['static_test_data'] = fold_test
        
        # Initialize and train model
        model = VAE(VAE_config, **kw_dict)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        print(f"Fold shapes - Train: {fold_train.shape}, Test: {fold_test.shape}")
        
        # Training loop
        best_hr10 = 0
        for epoch in range(50):
            # Тренируем только на данных взаимодействий (без pos/neg столбцов)
            loss = model.train_one_epoch(fold_train.iloc[:, :num_items].to_numpy(), 
                                       optimizer, batch_size=128)
            
            # Evaluation
            model.eval()
            with torch.no_grad():
                hit_rate_at_10, _, _, _, _ = recommender_evaluations(model, **fold_kw_dict)
                
                if hit_rate_at_10 > best_hr10:
                    best_hr10 = hit_rate_at_10
        
        fold_results.append(best_hr10)
        print(f"Fold {fold + 1} best HR@10: {best_hr10:.4f}")
    
    print("\nCross-validation results:")
    print(f"Mean HR@10: {np.mean(fold_results):.4f} ± {np.std(fold_results):.4f}")
    return fold_results

In [13]:
train_losses_dict = {}
test_losses_dict = {}
HR10_dict = {}

def VAE_objective(trial):
    lr = trial.suggest_float('learning_rate', 0.001, 0.01)
    batch_size = trial.suggest_categorical('batch_size', [64,128,256])
    epochs = 50  # Increased number of epochs to find optimal value
    
    if data_name == "Pinterest":
        model = VAE(Pinterest_VAE_config, **kw_dict)
    else:
        model = VAE(VAE_config, **kw_dict)
        
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_losses = []
    test_losses = []
    hr10 = []
    best_hr10 = 0
    epochs_without_improvement = 0
    
    print('======================== new run ========================')
    logger.info('======================== new run ========================')
    
    for epoch in range(epochs):
        # Learning rate decay
        if epoch!=0 and epoch%10 == 0:
            lr = 0.1*lr
            optimizer.lr = lr
            
        # Training step
        loss = model.train_one_epoch(train_array, optimizer, batch_size)
        train_losses.append(loss)
        
        # Save checkpoint at each epoch
        torch.save(model.state_dict(), 
                  Path(checkpoints_path, f'VAE_{data_name}_{trial.number}_{epoch}_{round(lr,4)}_{batch_size}.pt'))

        # Evaluation step
        model.eval()
        test_matrix = static_test_data.to_numpy()
        test_tensor = torch.Tensor(test_matrix[:,:-2]).to(device)
        test_pos = test_array[:,-2]
        test_neg = test_array[:,-1]
        row_indices = np.arange(test_matrix.shape[0])
        test_tensor[row_indices,test_pos] = 0
        output = model(test_tensor).to(device)
        pos_loss = -output[row_indices,test_pos].mean()
        neg_loss = output[row_indices,test_neg].mean()
        print(f'Epoch {epoch} - pos_loss = {pos_loss}, neg_loss = {neg_loss}')
        
        # Calculate metrics
        hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR = recommender_evaluations(model, **kw_dict)
        hr10.append(hit_rate_at_10)
        print(f'Epoch {epoch} - HR@10: {hit_rate_at_10:.4f}, HR@50: {hit_rate_at_50:.4f}, '
              f'HR@100: {hit_rate_at_100:.4f}, MRR: {MRR:.4f}, MPR: {MPR:.4f}')
        
        test_losses.append(pos_loss.item())
        
        # Track improvements in HR@10
        if hit_rate_at_10 > best_hr10:
            best_hr10 = hit_rate_at_10
            best_epoch = epoch
            epochs_without_improvement = 0
            # Save the best model separately
            torch.save(model.state_dict(), 
                      Path(checkpoints_path, f'VAE_{data_name}_{trial.number}_best_{round(lr,4)}_{batch_size}.pt'))
        else:
            epochs_without_improvement += 1
        
        # Early stopping if no improvement for 10 epochs
        if epochs_without_improvement >= 10:
            logger.info(f'Early stop at epoch {epoch}. Best HR@10: {best_hr10:.4f} at epoch {best_epoch}')
            break
    
    logger.info(f'Trial {trial.number} finished. Best HR@10: {best_hr10:.4f} at epoch {best_epoch}')
    train_losses_dict[trial.number] = train_losses
    test_losses_dict[trial.number] = test_losses
    HR10_dict[trial.number] = hr10
    
    return best_hr10

def analyze_checkpoints(best_trial_num):
    """
    Analyzes the best trial to determine gold, silver, and bronze checkpoints
    based on model performance at different epochs.
    
    Args:
        best_trial_num: The number of the best trial from optimization
        
    Returns:
        tuple: Epochs for gold, silver, and bronze checkpoints
    """
    hr10_values = HR10_dict[best_trial_num]
    max_hr10 = max(hr10_values)
    best_epoch = hr10_values.index(max_hr10)
    
    # Gold - best performing epoch
    gold_hr10 = max_hr10
    gold_epoch = best_epoch
    
    # Silver - ~60% of maximum performance
    silver_target = 0.6 * max_hr10
    silver_epoch = next(i for i, x in enumerate(hr10_values) 
                       if x >= silver_target)
    silver_hr10 = hr10_values[silver_epoch]
    
    # Bronze - early epoch with lower performance
    bronze_epoch = min(5, len(hr10_values)-1)  # take epoch 5 or earlier
    bronze_hr10 = hr10_values[bronze_epoch]
    
    print("\nCheckpoint Analysis:")
    print(f"Gold   - Epoch {gold_epoch}, HR@10: {gold_hr10:.4f}")
    print(f"Silver - Epoch {silver_epoch}, HR@10: {silver_hr10:.4f}")
    print(f"Bronze - Epoch {bronze_epoch}, HR@10: {bronze_hr10:.4f}")
    
    return gold_epoch, silver_epoch, bronze_epoch

In [14]:
def plot_training_progress(evaluation_metrics):
    plt.figure(figsize=(12, 6))
    
    if 'HR@10' in evaluation_metrics:
        for trial_num, hr_values in evaluation_metrics['HR@10'].items():
            if hr_values:  # Проверяем, что есть значения
                plt.plot(range(len(hr_values)), hr_values, 
                        label=f'Trial {trial_num}')
                print(f"Plotting trial {trial_num} with {len(hr_values)} values. "
                      f"Max HR@10: {max(hr_values):.4f}")
    
    plt.xlabel('Epoch')
    plt.ylabel('HR@10')
    plt.title('HR@10 over epochs')
    plt.grid(True)
    plt.legend()
    plt.savefig(f'VAE_metrics_{data_name}.png')
    plt.close()

In [15]:
def analyze_convergence(evaluation_metrics):
    best_epochs = []
    best_values = []
    
    for trial_values in evaluation_metrics['HR@10'].values():
        best_value = max(trial_values)
        best_epoch = trial_values.index(best_value)
        best_epochs.append(best_epoch)
        best_values.append(best_value)
    
    print(f"Statistics of best results achievement:")
    print(f"Mean epoch to best result: {np.mean(best_epochs):.1f}")
    print(f"Median epoch to best result: {np.median(best_epochs):.1f}")
    print(f"Mean best HR@10: {np.mean(best_values):.4f}")
    print(f"Best HR@10: {max(best_values):.4f}")
    print(f"Worst HR@10: {min(best_values):.4f}")

In [16]:
def analyze_results(evaluation_metrics):
    if not evaluation_metrics or not any(evaluation_metrics.values()):
        print("No data to analyze")
        return {}

    plt.figure(figsize=(15, 10))
    
    # График для каждой метрики
    for i, (metric_name, trials_data) in enumerate(evaluation_metrics.items(), 1):
        if not trials_data:  # Если нет данных для этой метрики
            continue
            
        plt.subplot(2, 3, i)
        for trial_num, values in trials_data.items():
            if values:  # Проверяем, что есть значения
                plt.plot(values, label=f'Trial {trial_num}')
        plt.title(f'{metric_name} over epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric_name)
        plt.grid(True)
        if i == 1:
            plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'VAE_metrics_{data_name}.png')
    plt.close()
    
    # Сохраняем сводную статистику только для метрик с данными
    summary_stats = {}
    for metric, trials_data in evaluation_metrics.items():
        if trials_data and any(trials_data.values()):
            summary_stats[metric] = {
                'best_value': max(max(values) if values else float('-inf') 
                                for values in trials_data.values()),
                'best_trial': max(trials_data.keys(),
                                key=lambda x: max(trials_data[x]) if trials_data[x] else float('-inf')),
                'mean_best': np.mean([max(values) if values else float('-inf') 
                                    for values in trials_data.values()]),
                'std_best': np.std([max(values) if values else float('-inf') 
                                  for values in trials_data.values()])
            }
    
    if summary_stats:  # Сохраняем только если есть данные
        with open(f'VAE_summary_stats_{data_name}.json', 'w') as f:
            json.dump(summary_stats, f, indent=4)
    
    return summary_stats

In [17]:
def analyze_convergence(evaluation_metrics):
    best_epochs = []
    best_values = []
    
    for trial_values in evaluation_metrics['HR@10'].values():
        best_value = max(trial_values)
        best_epoch = trial_values.index(best_value)
        best_epochs.append(best_epoch)
        best_values.append(best_value)
    
    print(f"Statistics of best results achievement:")
    print(f"Mean epoch to best result: {np.mean(best_epochs):.1f}")
    print(f"Median epoch to best result: {np.median(best_epochs):.1f}")
    print(f"Mean best HR@10: {np.mean(best_values):.4f}")
    print(f"Best HR@10: {max(best_values):.4f}")
    print(f"Worst HR@10: {min(best_values):.4f}")

In [18]:
def plot_training_progress(evaluation_metrics):
    plt.figure(figsize=(15, 10))
    
    # График для HR@10
    plt.subplot(2, 2, 1)
    for trial_num, hr_values in evaluation_metrics['HR@10'].items():
        if hr_values:  # Проверяем, что есть значения
            plt.plot(range(len(hr_values)), hr_values, 
                    label=f'Trial {trial_num}')
    plt.xlabel('Epoch')
    plt.ylabel('HR@10')
    plt.title('HR@10 over epochs')
    plt.grid(True)
    
    # Добавляем среднее значение по всем trials
    epochs_max = max(len(v) for v in evaluation_metrics['HR@10'].values())
    mean_hr = np.zeros(epochs_max)
    std_hr = np.zeros(epochs_max)
    for epoch in range(epochs_max):
        values = [v[epoch] for v in evaluation_metrics['HR@10'].values() 
                 if epoch < len(v)]
        mean_hr[epoch] = np.mean(values)
        std_hr[epoch] = np.std(values)
    
    plt.plot(range(epochs_max), mean_hr, 'k--', 
             linewidth=2, label='Mean HR@10')
    plt.fill_between(range(epochs_max), 
                     mean_hr - std_hr, 
                     mean_hr + std_hr, 
                     color='gray', alpha=0.2)
    
    # Отмечаем лучшую эпоху
    best_epoch = np.argmax(mean_hr)
    plt.plot(best_epoch, mean_hr[best_epoch], 'r*', markersize=15,
            label=f'Best: {mean_hr[best_epoch]:.4f} at epoch {best_epoch}')
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Добавляем статистику
    stats_text = (f"Final mean HR@10: {mean_hr[-1]:.4f}\n"
                 f"Best mean HR@10: {np.max(mean_hr):.4f}\n"
                 f"Best epoch: {best_epoch}")
    plt.text(1.1, 0.5, stats_text, transform=plt.gca().transAxes)

    plt.tight_layout()
    plt.savefig(f'VAE_metrics_{data_name}.png', bbox_inches='tight')
    plt.close()
    
    # Выводим статистику в консоль
    print("\nTraining progress statistics:")
    print(f"Number of trials: {len(evaluation_metrics['HR@10'])}")
    print(f"Average number of epochs per trial: {np.mean([len(v) for v in evaluation_metrics['HR@10'].values()]):.1f}")
    print(f"Best mean HR@10: {np.max(mean_hr):.4f} at epoch {best_epoch}")

## NCF training functions

In [19]:
train_losses_dict = {}
test_losses_dict = {}
HR10_dict = {}

## PAY ATTENTION to define manualy the MLP_model and GMF_model checkpoints which will be used inside the NCF

def NCF_objective(trial):
    lr = trial.suggest_float('learning_rate', 0.0005, 0.005)
    batch_size = trial.suggest_categorical('batch_size', [32,64,128])
    beta = trial.suggest_float('beta',0, 4)
    epochs = 20
    MLP = MLP_model(hidden_size=8, num_layers=3, **kw_dict)
    # EDIT HERE
    MLP_checkpoint = torch.load(Path(checkpoints_path, 'MLP_model_ML1M_0.0001_64_27.pt'))
    MLP.load_state_dict(MLP_checkpoint)
    MLP.train()
    GMF = GMF_model(hidden_size=8, **kw_dict)
    # & EDIT HERE
    GMF_checkpoint = torch.load(Path(checkpoints_path, 'GMF_best_ML1M_0.0001_32_17.pt'))
    GMF.load_state_dict(GMF_checkpoint)
    GMF.train()
    model = NCF(factor_num=8, num_layers=3, dropout=0.5, model= 'NeuMF-pre', GMF_model= GMF, MLP_model=MLP, **kw_dict)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_losses = []
    test_losses = []
    hr10 = []
    print(f'======================== new run - {recommender_name} ========================')
    logger.info(f'======================== new run - {recommender_name} ========================')
    
    num_training = train_data.shape[0]
    num_batches = int(np.ceil(num_training / batch_size))

    
    for epoch in range(epochs):
        train_matrix = sample_indices(train_data.copy(), **kw_dict)
        perm = np.random.permutation(num_training)
        loss = []
        train_pos_loss=[]
        train_neg_loss=[]
        if epoch!=0 and epoch%10 == 0:
            lr = 0.1*lr
            optimizer.lr = lr
        
        for b in range(num_batches):
            optimizer.zero_grad()
            if (b + 1) * batch_size >= num_training:
                batch_idx = perm[b * batch_size:]
            else:
                batch_idx = perm[b * batch_size: (b + 1) * batch_size]    
            batch_matrix = torch.FloatTensor(train_matrix[batch_idx,:-2]).to(device)

            batch_pos_idx = train_matrix[batch_idx,-2]
            batch_neg_idx = train_matrix[batch_idx,-1]
            
            batch_pos_items = torch.Tensor(items_array[batch_pos_idx]).to(device)
            batch_neg_items = torch.Tensor(items_array[batch_neg_idx]).to(device)
            
            pos_output = model(batch_matrix, batch_pos_items)
            neg_output = model(batch_matrix, batch_neg_items)

            pos_loss = -torch.log(pos_output).mean()
            neg_loss = -torch.log(torch.ones_like(neg_output)-neg_output).mean()

            batch_loss = pos_loss + beta*neg_loss
            if batch_loss<torch.inf:
                batch_loss.backward()
                optimizer.step()
            
            loss.append(batch_loss.item())
            train_pos_loss.append(pos_loss.item())
            train_neg_loss.append(neg_loss.item())
            
        print(f'train pos_loss = {np.mean(train_pos_loss)}, neg_loss = {np.mean(train_neg_loss)}')    
        train_losses.append(np.mean(loss))
        torch.save(model.state_dict(), Path(checkpoints_path, f'{recommender_name}2_{data_name}_{round(lr,5)}_{batch_size}_{trial.number}_{epoch}.pt'))


        model.eval()
        test_matrix = np.array(static_test_data)
        test_tensor = torch.Tensor(test_matrix[:,:-2]).to(device)
        
        test_pos = test_matrix[:,-2]
        test_neg = test_matrix[:,-1]
        
        row_indices = np.arange(test_matrix.shape[0])
        test_tensor[row_indices,test_pos] = 0
        
        pos_items = torch.Tensor(items_array[test_pos]).to(device)
        neg_items = torch.Tensor(items_array[test_neg]).to(device)
        
        pos_output = model(test_tensor, pos_items).to(device)
        neg_output = model(test_tensor, neg_items).to(device)
        
        pos_loss = -torch.log(pos_output).mean()
        neg_loss = -torch.log(torch.ones_like(neg_output)-neg_output).mean()
        print(f'test pos_loss = {pos_loss}, neg_loss = {neg_loss}')
        
        hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR = recommender_evaluations(model, **kw_dict)
        hr10.append(hit_rate_at_10)
        print(hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR)
                   
        
        test_losses.append(-hit_rate_at_10)
        if epoch>5:
            if test_losses[-2]<=test_losses[-1] and test_losses[-3]<=test_losses[-2] and test_losses[-4]<=test_losses[-3]:
                logger.info(f'Early stop at trial with batch size = {batch_size} and lr = {lr}. Best results at epoch {np.argmin(test_losses)} with value {np.min(test_losses)}')
                train_losses_dict[trial.number] = train_losses
                test_losses_dict[trial.number] = test_losses
                HR10_dict[trial.number] = hr10
                return max(hr10)
            
    logger.info(f'Stop at trial with batch size = {batch_size} and lr = {lr}. Best results at epoch {np.argmin(test_losses)} with value {np.min(test_losses)}')
    train_losses_dict[trial.number] = train_losses
    test_losses_dict[trial.number] = test_losses
    HR10_dict[trial.number] = hr10
    return max(hr10)

In [20]:
logger = logging.getLogger()

logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler(f"{recommender_name}_{data_name}_Optuna.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(direction='maximize')

logger.info("Start optimization.")
study.optimize(MLP_objective, n_trials=20)

with open(f"{recommender_name}_{data_name}_Optuna.log") as f:
    assert f.readline().startswith("A new study created")
    assert f.readline() == "Start optimization.\n"
    
    
# Print best hyperparameters and corresponding metric value
print("Best hyperparameters: {}".format(study.best_params))
print("Best metric value: {}".format(study.best_value))

======================== new run - MLP ========================
train pos_loss = 0.395270636677742, neg_loss = 0.16510765701532365
test pos_loss = 0.5951240062713623, neg_loss = 0.08138133585453033
Debug shapes in recommender_evaluations:
static_test_data shape: (1208, 3383)
items_array shape: (3381, 3381)
num_items: 3381
0.005794701986754967 0.054635761589403975 0.09271523178807947 0.0008278145695364238 49.7377975088682
train pos_loss = 0.6210036873817444, neg_loss = 0.07202535718679429
test pos_loss = 0.5802581906318665, neg_loss = 0.0893411636352539
Debug shapes in recommender_evaluations:
static_test_data shape: (1208, 3383)
items_array shape: (3381, 3381)
num_items: 3381
0.0380794701986755 0.07119205298013245 0.09602649006622517 0.0008278145695364238 43.60665659871781
train pos_loss = 0.5720695495605469, neg_loss = 0.09119329750537872
test pos_loss = 0.5581821799278259, neg_loss = 0.09324529021978378
Debug shapes in recommender_evaluations:
static_test_data shape: (1208, 3383)
ite

# Evaluations

## Load the trained recommender

In [21]:
recommender_path_dict = {
    ("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0007_128_10.pt"),
    ("ML1M","MLP"):Path(checkpoints_path, "MLP_ML1M_0.002_1024_19_8.pt"),
    ("ML1M","NCF"):Path(checkpoints_path, "NCF_ML1M_5e-05_64_16.pt"),

    ("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_0.0001_128_13.pt"),
    ("Yahoo","MLP"):Path(checkpoints_path, "MLP2_Yahoo_0.0083_128_1.pt"),
    ("Yahoo","NCF"):Path(checkpoints_path, "NCF_Yahoo_0.001_64_21_0.pt"),
    
    ("Pinterest","VAE"): Path(checkpoints_path, "VAE_Pinterest_12_18_0.0001_256.pt"),
    ("Pinterest","MLP"):Path(checkpoints_path, "MLP_Pinterest_0.0062_512_21_0.pt"),
    ("Pinterest","NCF"):Path(checkpoints_path, "NCF2_Pinterest_9e-05_32_9_10.pt")}


hidden_dim_dict = {
    ("ML1M","VAE"): None,
    ("ML1M","MLP"): 32,
    ("ML1M","NCF"): 8,
    
    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    ("Yahoo","NCF"):8,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512,
    ("Pinterest","NCF"): 64}

In [22]:
hidden_dim = hidden_dim_dict[(data_name, recommender_name)]
recommender_path = recommender_path_dict[(data_name, recommender_name)]

In [23]:
model = load_recommender(data_name, hidden_dim, checkpoints_path, recommender_path, **kw_dict)

TypeError: super(type, obj): obj must be an instance or subtype of type

## plot the distribution of top recommended item accross all users

In [ ]:
# plot the distribution of top recommended item accross all users
topk_train = {}
for i in range(len(train_array)):
    vec = train_array[i]
    tens = torch.Tensor(vec).to(device)
    topk_train[i] = int(get_user_recommended_item(tens, model).cpu().detach().numpy())

In [ ]:
plt.hist(topk_train.values(), bins=1000)
plt.plot(np.array(list(pop_dict.keys())), np.array(list(pop_dict.values()))*100, alpha=0.2)
plt.show()

In [ ]:
topk_test = {}
for i in range(len(test_array)):
    vec = test_array[i]
    tens = torch.Tensor(vec).to(device)
    topk_test[i] = int(get_user_recommended_item(tens, model).cpu().detach().numpy())

In [ ]:
plt.hist(topk_test.values(), bins=400)
plt.plot(np.array(list(pop_dict.keys())), np.array(list(pop_dict.values()))*200, alpha=0.2)
plt.show() 

In [ ]:
hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR = recommender_evaluations(model)

In [ ]:
print(hit_rate_at_10, hit_rate_at_50, hit_rate_at_100, MRR, MPR)